### Parallel Monte Carlo

In this fully parallelized Monte Carlo simulation all workers share the same network structure, but are 
given different attacks at different entrypoint at each step. 

#### Demonstration: Writing Many Tasks to Same Array
This is a minimal implementation in the Ray multiprocessing library (Core module) to write many tasks to one common array. This forms the scaffolding for parallelizing the Monte Process.

In [ ]:
import numpy as np
import tqdm
import ray
from ray.util.queue import Queue
from dataclasses import dataclass

@ray.remote
class MonteOverseer():
    
    def __init__(self, n_entrypoints, n_attacks_per_entrypoint, queue:Queue):
        self.fill_level = 0
        self.fill_limit = n_entrypoints*n_attacks_per_entrypoint
        self.results = np.zeros((n_entrypoints, n_attacks_per_entrypoint), dtype=np.float32)
        self.queue = queue
        
    def get(self):
        return self.results
    
    def run(self):
        while True:
            (result, i, j) = queue.get(block=True)
            self.results[i, j] = result
            self.fill_level += 1
            # Finished processing all tasks
            if self.fill_level == self.fill_limit:
                break

@ray.remote(num_cpus=1)
class MonteActor():
    
    """
    Each Monte Actor runs in its own Python process.
    """
    
    def __init__(self, queue:Queue):
        self.i = 0
        self.queue = queue
    
    def work(self, idx, seed, entrypoint):
        self.i += 1
        self.queue.put_nowait((self.i, idx, entrypoint))
        return self.i
    
    def get(self):
        return self.i
    
    def reset(self):
        pass

if not ray.is_initialized():
    ray.init()
n_cpu = int(ray.available_resources().get("CPU", 1))
N = 100
vary_entrypoint = True
N_ENTRYPOINTS = 98
GLOBAL_SEED = 0
np.random.seed(GLOBAL_SEED)

queue = Queue()
overseer = MonteOverseer.remote(N, N_ENTRYPOINTS, queue)
workers = [MonteActor.remote(queue) for _ in range(n_cpu-1)]


idcs = np.arange(N)
seeds = np.random.choice(N, size=N, replace=False)
entrypoints = np.arange(N_ENTRYPOINTS)
overseer.run.remote()
pos = 0
for entrypoint in tqdm.tqdm(entrypoints, desc="Entrypoint :: ", total=N_ENTRYPOINTS, position=0):
    for idx, seed in tqdm.tqdm(zip(idcs, seeds), desc="Iteration ::", total=N, position=1):
        workers[pos % len(workers)].work.remote(idx, seed, entrypoint)
        pos += 1
        
print(ray.get(overseer.get.remote()))

In [ ]:
import inspect, warnings, random, copy
import numpy as np
import pandapower
import tqdm
import ray
from pathlib import Path
from ray.util.queue import Queue
from dataclasses import dataclass
from attackers.interface import Attacker
from attackers.random_attacker import RandomAttacker
from communication.network import CommNetwork
from communication.components import Device
from cyber.criticality import criticality_by_capacity


@dataclass
class AttackerConfig:
    budget:float 
    auto_compromise_children:bool = False
    verbose:bool = False
    
@dataclass
class MonteSample:
    attack_idx:int
    entrypoint_idx:int
    n_compromised:int
    total_effort_spent:float
    critical_sum:float = 0.0

@ray.remote
class MonteOverseer():
    
    def __init__(self, n_entrypoints, n_attacks_per_entrypoint, queue:Queue):
        self.fill_level = 0
        self.fill_limit = n_entrypoints*n_attacks_per_entrypoint
        self.compromised = np.zeros((n_entrypoints, n_attacks_per_entrypoint), dtype=np.int32)
        self.effort = np.zeros((n_entrypoints, n_attacks_per_entrypoint), dtype=np.float32)
        self.criticality = np.zeros((n_entrypoints, n_attacks_per_entrypoint), dtype=np.float32)
        self.queue = queue
        
    def get(self):
        return (self.compromised, self.effort, self.criticality)
    
    def run(self):
        while True:
            sample:MonteSample = queue.get(block=True)
            
            # Put sample into pre-allocated arrays
            i, j = sample.entrypoint_idx, sample.attack_idx
            self.compromised[i, j] = sample.n_compromised
            self.effort[i, j] = sample.total_effort_spent
            self.criticality[i, j] = sample.critical_sum
            
            self.fill_level += 1
            # Finished processing all tasks
            if self.fill_level == self.fill_limit:
                break

@ray.remote(num_cpus=1)
class MonteActor():
    
    """
    Each Monte Actor runs in its own Python process.
    """
    
    def __init__(self, queue:Queue, global_seed:int,
                 attacker_class:Attacker,
                 attacker_config:AttackerConfig, 
                 device_only:bool=True,
                 **network_kwargs):
        self.i = 0
        self.queue = queue
        # NOTE: Each time the network is initialized, the effort and success prob. is fixed
        # Need to reset these each time to ensure we get different numbers when compromising
        # it again
        self.network_kwargs = network_kwargs
        self.attacker_kwargs = attacker_config.__dict__
        random.seed(global_seed)
        np.random.seed(global_seed)
        self.pcn = CommNetwork(**network_kwargs)
        self.attacker = attacker_class(**attacker_config.__dict__)
        self.device_only = device_only
        
    def work(self, idx:int, seed:int, entrypoint:int):
        self.i += 1
        random.seed(seed)
        np.random.seed(seed)
        self.pcn.reset(entrypoint)
        # self.pcn.set_entrypoints(entrypoint)
        nodes_compromised, total_effort_spent = self.attacker.attack_network(self.pcn)
        # Count how many nodes were compromised (and add up their criticality)
        critical_sum, device_count = 0, 0
        for n in nodes_compromised:
            if isinstance(n, Device):
                critical_sum += n.equipment.criticality if n.equipment is not None else 0
                device_count += 1
        
        # Package results as a MonteSample dataclass
        n_compromised = device_count if self.device_only else len(nodes_compromised)
        sample = MonteSample(idx, entrypoint, n_compromised, total_effort_spent, critical_sum)
        
        # Asynchronously publish results to queue
        self.queue.put_nowait(sample)

    def get(self):
        return self.i
    
if not ray.is_initialized():
    ray.init()
n_cpu = int(ray.available_resources().get("CPU", 1)) - 4
N = 10
vary_entrypoint = True
N_ENTRYPOINTS = 2
GLOBAL_SEED = 0
BUDGET = 52.0
AUTO_COMP_CHILDREN = False
GRID_KWARGS = {"with_der":"all"}
CRITICALITY = criticality_by_capacity
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# Get Grid
grid = "create_cigre_network_mv"
grid_map = {name:creator for name, creator in inspect.getmembers(pandapower.networks, predicate=inspect.isfunction)}
with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", category=FutureWarning)
    if type(grid) is str:
        grid = grid_map.get(grid)(**GRID_KWARGS) if grid is not None else None
    else:
        grid = pandapower.from_json(grid)

# Get Criticality

criticality = CRITICALITY(grid, verbose=False)[0] if CRITICALITY is not None else CRITICALITY

queue = Queue()
overseer = MonteOverseer.remote(N_ENTRYPOINTS, N, queue)
attacker_config = AttackerConfig(budget=BUDGET,
                                 auto_compromise_children=AUTO_COMP_CHILDREN,
                                 verbose=False)
network_params = dict(
    n_devices=20,
    children_per_parent=3, child_no_deviation=2,
    network_specs=Path.cwd() / "specifications" / "Default_specifications.json",
    grid=grid,
    criticality=criticality,
    crit_norm=False,
    effort_only=False,
    sibling_to_sibling_comm="all",
    n_entrypoints=1,
)
workers = [MonteActor.remote(queue, GLOBAL_SEED, RandomAttacker, attacker_config,
                             **network_params) for _ in range(n_cpu-1)]


idcs = np.arange(N)
seeds = np.random.choice(N, size=N, replace=False)
print(seeds)
entrypoints = np.arange(N_ENTRYPOINTS)
overseer.run.remote()
pos = 0
for entrypoint in tqdm.tqdm(entrypoints, desc="Entrypoint :: ", total=N_ENTRYPOINTS, position=0):
    for idx, seed in tqdm.tqdm(zip(idcs, seeds), desc="Iteration ::", total=N, position=1):
        workers[pos % len(workers)].work.remote(idx, int(seed), entrypoint)
        pos += 1

compromised, effort, criticality = ray.get(overseer.get.remote())
print(compromised)
print(effort)
print(criticality)